In [6]:
import pandas as pd
df = pd.read_feather('all_merged_99.feather')
df.info()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


<class 'pandas.core.frame.DataFrame'>
Index: 5964147 entries, 0 to 5964154
Columns: 889 entries, MaskID to howmanydoseshastheparticipantm
dtypes: Int64(233), category(470), float16(182), float64(3), int64(1)
memory usage: 16.7 GB


In [7]:
!df -h

Filesystem                                                       Size  Used Avail Use% Mounted on
devtmpfs                                                          35G     0   35G   0% /dev
tmpfs                                                             35G     0   35G   0% /dev/shm
tmpfs                                                             35G  816K   35G   1% /run
tmpfs                                                             35G     0   35G   0% /sys/fs/cgroup
/dev/nvme0n1p1                                                   135G   87G   49G  64% /
/dev/nvme1n1                                                      20G   15G  4.1G  79% /home/ec2-user/SageMaker
tmpfs                                                            6.9G     0  6.9G   0% /run/user/1002
tmpfs                                                            6.9G     0  6.9G   0% /run/user/1001
tmpfs                                                            6.9G     0  6.9G   0% /run/user/0
tmpfs           

# 1. Columns of Date

#### Steps of date column harmonization:
1. Filter out date column sets based on their headers
2. Create one date column to replace one set of date columns.
3. assign values in the format of date_time to the new created column.

## 1.1 Day, Month, Year

In [8]:
# Filter all columns with "Day", "Month", "Year" as date sets
# in lower case for case-insensitive comparison
# Example strings to include
contain_list_1 = ['Day', "Month", 'Year'] 
# Example strings to exclude
not_contain_list_1 = ['LastYear', 'Last6Months', '3Days', 'Birth', 'MostRecentHbA1c', 'PastYear', 
                      'ChangeInDrugStatusEffective', 'ParticipantDateDispensed', 'MechanisticSampleCollection',
                     'Polydipsia', 'Polyuria']     

# Converting each string in the lists to lower case
contain_list_1_low = [string.lower() for string in contain_list_1]
not_contain_list_1_low = [string.lower() for string in not_contain_list_1]

# Filtering columns with case-insensitive comparison
filtered_date_columns_1 = [col for col in df.columns if 
                    any(contain in col.lower() for contain in contain_list_1_low) and 
                    all(not_contain not in col.lower() for not_contain in not_contain_list_1_low)]

print("Found ", len(filtered_date_columns_1), "columns: ", filtered_date_columns_1)

filtered_df = df[filtered_date_columns_1]

# Output the filtered DataFrame
print(filtered_df.info())

# Uncomment if you want to print top 20 values for each column
#for col in filtered_columns:
#    print(f"Top 20 values for column {col}:")
#    print(df[col].value_counts().nlargest(20))
#    print()

Found  102 columns:  ['_15Aug2011ConsentDateDay', '_15Aug2011ConsentDateMonth', '_15Aug2011ConsentDateYear', 'Fall2017ConsentDateDay', 'Fall2017ConsentDateMonth', 'Fall2017ConsentDateYear', 'DateOfBloodDrawDay', 'DateOfBloodDrawMonth', 'DateOfBloodDrawYear', 'DateReportedDay', 'DateReportedMonth', 'DateReportedYear', 'ScreeningDay', 'ScreeningMonth', 'ScreeningYear', 'ConsentSignedDay', 'ConsentSignedMonth', 'ConsentSignedYear', 'Spring2019ConsentDateMonth_dup12', 'Spring2019ConsentDateDay_dup12', 'Spring2019ConsentDateYear_dup12', 'DateConsentSignedDay', 'DateConsentSignedMonth', 'DateConsentSignedYear', 'SamplesCollectionDay', 'SamplesCollectionMonth', 'SamplesCollectionYear', 'SamplesCollectionDay_dup15', 'SamplesCollectionMonth_dup15', 'SamplesCollectionYear_dup15', 'DateConsentSignedDay_dup15', 'DateConsentSignedMonth_dup15', 'DateConsentSignedYear_dup15', 'DateStatusChangeDay', 'DateStatusChangeMonth', 'DateStatusChangeYear', 'ProtcolDeviationDay', 'ProtcolDeviationMonth', 'Protc

In [9]:
#check columns for the day
day_column_list_1 = [col for col in filtered_date_columns_1 if "Day" in col]
print("No. of the day columes to be merged: ", len(day_column_list_1))

#create columns for the month and the year based on columns of the day
month_column_list_1 = [name.replace("Day", "Month") for name in day_column_list_1]
year_column_list_1 = [name.replace("Day", "Year") for name in day_column_list_1]

#check whether there are any columns in the dataframe not following the naming rules
unmatched_month_columns = [name for name in month_column_list_1 if name not in df.columns]
print("No. of month columes in wrong naming format: ", len(unmatched_month_columns))

unmatched_year_columns = [name for name in year_column_list_1 if name not in df.columns]
print("No. of year columes in wrong naming format: ", len(unmatched_year_columns))

No. of the day columes to be merged:  34
No. of month columes in wrong naming format:  0
No. of year columes in wrong naming format:  0


In [10]:
import numpy as np
# Convert 'NA' strings to np.nan
def convert_na(value):
    if pd.isna(value):  # using pandas function to check for NA
        return np.nan   # converting to numpy NaN
    else:
        return value

# Function to convert month from string or number to numeric month, handling missing values
def month_to_num(month):
    if pd.isna(month):
        return np.nan
    try:
        # Handle string representation
        if isinstance(month, str):
            datetime_object = pd.to_datetime(month, format='%b')
            return datetime_object.month
        # Handle numeric representation
        elif isinstance(month, (int, float)):
            return int(month)
    except ValueError:
        return np.nan
    
# Create a date column and assign values by combine date columns with similar header
def create_date_column(df, year_col, original_cols):
    parts = year_col.split('Year')
    base_col_name = parts[0]
    suffix = parts[1] if len(parts) > 1 else ''

    day_col = base_col_name + 'Day' + suffix
    month_col = base_col_name + 'Month' + suffix

    if day_col in original_cols and month_col in original_cols:
        date_col = base_col_name + 'Date' + suffix
        problematic_rows = []
        for index, row in df.iterrows():
            try:
                if pd.isna(row[year_col]) or pd.isna(row[month_col]) or pd.isna(row[day_col]):
                    df.at[index, date_col] = np.nan
                else:
                    year = int(row[year_col])
                    month = int(month_to_num(row[month_col]))
                    day = int(row[day_col])
                    datetime_obj = pd.to_datetime(f"{year}-{month:02d}-{day:02d}", errors='coerce')
                    df.at[index, date_col] = datetime_obj.date()  # Extracting just the date part
            except (ValueError, OverflowError) as e:
                problematic_rows.append((index, row[year_col], row[month_col], row[day_col]))
                df.at[index, date_col] = np.nan
        if problematic_rows:
            print("Problematic rows:")
            for row in problematic_rows:
                print(row)
    else:
        print(f"Matching columns for '{year_col}' not found.")


In [ ]:
# Convert 'NA' strings to np.nan
for col in filtered_date_columns_1:
    filtered_df.loc[:, col] = filtered_df[col].apply(convert_na)

# Iterating over columns and creating date columns
for col in year_column_list_1:
    create_date_column(filtered_df, col, filtered_date_columns_1)

print(filtered_df.info())
filtered_df.head()

/tmp/ipykernel_9130/375426993.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, date_col] = datetime_obj.date()  # Extracting just the date part
/tmp/ipykernel_9130/375426993.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2015-07-01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, date_col] = datetime_obj.date()  # Extracting just the date part
/tmp/ipykernel_9130/375426993.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [ ]:
filtered_df.info()

## 1.2 Month, Year

'BirthMonth', 'BirthYear'
'MostRecentHbA1cMonth', 'MostRecentHbA1cYear'
'PolyuriaMonth', 'PolyuriaYear'
'PolydipsiaMonth', 'Polydipsiayear', 

## 1.3 DD, MM, YYYY

## 1.4 day, mon, yea